In [1]:
from tensorflow.keras import Sequential
from tensorflow.keras.datasets import reuters
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer

# 파일 불러오기

In [2]:
# 쥬피터용
# 데이터 셋 posneg.xlsx
# df = pd.read_csv('./도레미파솔 3차 크롤링/마지막 학습용 데이터/taste_df.csv', encoding='utf-16')

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
df = pd.read_csv('drive/My Drive/Colab Notebooks/3rd Project/3rd_Project_Data/taste_TrainData.csv', encoding='utf-16')
df

,Unnamed: 0,taste,customer_review
0,14,0.0,"2인분 시켜서 반찬 좀 2명 분 달랬더니 매번 하나만 챙겨주네, 짜증"
1,52,0.0,커피도 반만 들어있고ㅠ음식도 성의없어서 이젠 주문안하고 싶네요ㅠ
2,54,0.0,된장찌개와 돼지구이 쌈밥을 시켰습니다 예상도착시간이 되도 아무 연락이 없어 가게에 ...
3,96,0.0,요청사항은 안보시나봐요..ㅡ 사이다 시켰는데 콜라로 오고. 주문사항이랑 요청사항 확...
4,97,0.0,김치찌개가 부대찌개도아니고 상한맛같앗어요 다른건 다 맛잇었는데 신뢰가 확..ㅜ 작년...
...,...,...,...
471237,537917,1.0,양은 역시 많습니다 ~ 항상 남기네요 ~
471238,537919,1.0,"배달도 빠르고 친절하고 맛있어요! 요즘 보쌈이나 족발 배달시켜먹으면 너무 비싼데, ..."
471239,537920,1.0,자주시켜먹는곳입니다
471240,537921,1.0,이건 강추~ 다만 양이 넘 많이서 3병이서 먹엇는데도 많이 남앗습니다 ~ 막국수가 ...


In [8]:
df.columns

Index(['Unnamed: 0', 'taste', 'customer_review'], dtype='object')

In [9]:
df.head(5)

,Unnamed: 0,taste,customer_review
0,14,0.0,"2인분 시켜서 반찬 좀 2명 분 달랬더니 매번 하나만 챙겨주네, 짜증"
1,52,0.0,커피도 반만 들어있고ㅠ음식도 성의없어서 이젠 주문안하고 싶네요ㅠ
2,54,0.0,된장찌개와 돼지구이 쌈밥을 시켰습니다 예상도착시간이 되도 아무 연락이 없어 가게에 ...
3,96,0.0,요청사항은 안보시나봐요..ㅡ 사이다 시켰는데 콜라로 오고. 주문사항이랑 요청사항 확...
4,97,0.0,김치찌개가 부대찌개도아니고 상한맛같앗어요 다른건 다 맛잇었는데 신뢰가 확..ㅜ 작년...


In [10]:
len(df)

471242

In [11]:
neg_test = df[df['taste']==0]
len(neg_test)

21436

In [12]:
pos_test = df[df['taste']==1]
len(pos_test)

449806

In [15]:
pos_test = pos_test.sample(n=30000).reset_index(drop=True)
len(pos_test)

30000

In [18]:
df_TrainSet = pd.concat([pos_test,neg_test])
len(df_TrainSet)

51436

In [20]:
df_TrainSet = df_TrainSet.sample(frac=1).reset_index(drop=True)
df_TrainSet

,Unnamed: 0,taste,customer_review
0,479676,0.0,다 식어서 왔습니다. 푸라닭 조금 실망입니다. 배달 예상 시간 10분 이상 늦은 건...
1,284610,1.0,밥이 좀 질어진 것 빼고 괜찮았어요
2,288931,0.0,짬뽕맛이 아니에요 너무기름지고 아쉽네요ㅠㅠ
3,141287,1.0,바쁘실텐데 빨리 갖다주셔서 감사합니다~
4,473692,0.0,배달은 빠르지만 치킨은 식어있을땨가 많아요 미니스톱 치킨드세요
...,...,...,...
51431,78988,1.0,이게 뭐라고 며칠째 시켜대는 중
51432,419843,0.0,여기 만두는 냉동 만두에 북어국 당근이우동에두 있고 복어국 육수랑 우동 육수가 같은...
51433,288637,1.0,맛있어요 초밥 괜찮네용
51434,301101,0.0,떡뽁이도 그냥그렇고 김밥은 딱딱하고 비추


In [21]:
X = df_TrainSet['customer_review']

In [22]:
y = df_TrainSet['taste']

In [23]:
X

0        다 식어서 왔습니다. 푸라닭 조금 실망입니다. 배달 예상 시간 10분 이상 늦은 건...
1                                      밥이 좀 질어진 것 빼고 괜찮았어요
2                                  짬뽕맛이 아니에요 너무기름지고 아쉽네요ㅠㅠ
3                                    바쁘실텐데 빨리 갖다주셔서 감사합니다~
4                       배달은 빠르지만 치킨은 식어있을땨가 많아요 미니스톱 치킨드세요
                               ...                        
51431                                    이게 뭐라고 며칠째 시켜대는 중
51432    여기 만두는 냉동 만두에 북어국 당근이우동에두 있고 복어국 육수랑 우동 육수가 같은...
51433                                         맛있어요 초밥 괜찮네용
51434                               떡뽁이도 그냥그렇고 김밥은 딱딱하고 비추
51435                                        수고하셨습니다~（＾◇＾）
Name: customer_review, Length: 51436, dtype: object

In [24]:
y

0        0.0
1        1.0
2        0.0
3        1.0
4        0.0
        ... 
51431    1.0
51432    0.0
51433    1.0
51434    0.0
51435    1.0
Name: taste, Length: 51436, dtype: float64

# 시작

In [25]:
#KoNLPy패키지에는 Hannanum, Kkma, Komoran, Mecab, Okt(Twitter)등의 클래스들이 있습니다
# https://mr-doosun.tistory.com/22
# Okt (Twitter) Class
# 해당 분석기는 오픈 소스 한국어 분석기이고, 과거 트위터 형태소 분석기였습니다.

# KoNLPy 문서에 따른 클래스간의 성능 비교
# https://konlpy.org/ko/v0.4.3/morph/

# morphs  텍스트에서 형태소를 반환한다 / 형태소 : 일정한 의미가 있는 가장 작은 말의 단위
# nouns   텍스트에서 명사를 반환한다
# phrases 텍스트에서 어절을 뽑아낸다
# pos     텍스트에서 품사 정보를 부착하여 반환한다

In [26]:
pip install konlpy

     |████████████████████████████████| 19.4MB 1.2MB/s 
     |████████████████████████████████| 92kB 9.0MB/s 
     |████████████████████████████████| 3.8MB 25.4MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0


In [27]:
# okt(Open Korean Text) 
from konlpy.tag import Okt
okt = Okt()

In [28]:
# okt test
okt.morphs('KoNLPy패키지에는 Hannanum, Kkma, Komoran, Mecab, Okt(Twitter)등의 클래스들이 있습니다.')

['KoNLPy',
 '패키지',
 '에는',
 'Hannanum',
 ',',
 'Kkma',
 ',',
 'Komoran',
 ',',
 'Mecab',
 ',',
 'Okt',
 '(',
 'Twitter',
 ')',
 '등',
 '의',
 '클래스',
 '들',
 '이',
 '있습니다',
 '.']

In [29]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 84)

In [30]:
len(X_train),len(y_train), len(X_test), len(y_test)

(36005, 36005, 15431, 15431)

In [31]:
y_test

41173    0.0
45038    0.0
15472    0.0
19459    1.0
174      0.0
        ... 
49655    0.0
20944    1.0
8164     1.0
30348    1.0
26462    1.0
Name: taste, Length: 15431, dtype: float64

# TF-IDF (+파라미터)

In [33]:
# min-df DF(document-frequency)의 최소 빈도값 <-> max-df
# n-gram 것은 단어의 묶음
# analyzer = 'word'라고 설정시, 학습의 단위를 단어로 (ex - home, go, my ...)
# analyzer = 'char'라고 설정시, 학습의 단위를 글자로 (ex - a, b, c, d ...)
# ngram_range = (1, 2)라고 한다면, 단어의 묶음을 1개부터 2개까지 설정
# go home', 'very good'과 같은 2개 짜리 묶음도 인덱스를 받게 되는 것
# 단어가 묶여야 비로소 의미를 가지는 것들을 위해
# max_feature는 tf-idf vector의 최대 feature를 설정 /  사전에 EDA를 통해서 데이터의 length를 확인


from sklearn.feature_extraction.text import TfidfVectorizer
tfv = TfidfVectorizer(tokenizer=okt.morphs, ngram_range=(1,3), min_df=3, max_df=0.9)
tfv.fit(X_train)
tfv_X_train = tfv.transform(X_train)
tfv_X_train

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


<36005x63946 sparse matrix of type '<class 'numpy.float64'>'
	with 1082626 stored elements in Compressed Sparse Row format>

In [38]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

clf = LogisticRegression(random_state=0)
params = {'C': [1,3,5,7,9]}
grid_cv = GridSearchCV(clf, param_grid = params, cv=4, scoring='accuracy', verbose=1)
grid_cv.fit(tfv_X_train, y_train)

Fitting 4 folds for each of 5 candidates, totalling 20 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
 

GridSearchCV(cv=4, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=0, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=None, param_grid={'C': [1, 3, 5, 7, 9]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=1)

In [35]:
grid_cv.best_params_

{'C': 5}

In [36]:
grid_cv.best_score_

0.9142341123518306

In [37]:
tfv_X_test=tfv.transform(X_test)
grid_cv.best_estimator_.score(tfv_X_test,y_test)

0.9196422785302314

# 테스트

In [39]:
TestSet=pd.read_csv('/content/drive/My Drive/Colab Notebooks/3rd Project/3rd_Project_Data/test_taste_df.csv',encoding='utf-16')

In [66]:
neg_test = TestSet[TestSet['taste']==0]
print("부정: ", len(neg_test))
pos_test = TestSet[TestSet['taste']==1]
pos_test = pos_test.sample(n=500).reset_index(drop=True)
print("긍정: ", len(pos_test))
TestSet = pd.concat([neg_test, pos_test])
print("TestSet: ", len(TestSet))


부정:  405
긍정:  500
TestSet:  905


,Unnamed: 0,taste,customer_review
0,42,0.0,아구찜의 오징어가 간이 하나도 안됐고 주먹밥은 저번에 시켰을 때보다 양이 훨 적어졌...
1,74,0.0,90분이 넘었는데 배달이안와요 전화는 끊어버리고 어쩌라는겁니까~~~~~
2,79,0.0,이거뭡니까 전화해도 계속받지않고 추가한 반찬 오지않고 일부러 전화안받는겁니까 이런식...
3,108,0.0,맛은 괜찮은데 해산물이 싱싱하지 않아요 특히 게가 다 상했어요 맛이 이상해요ㅡㅡ
4,135,0.0,예상 배달 시간보다 늦어서 전화했는데 전화도 안받고 냉동이라 꽃게가 완전 죽이예요 ...


In [68]:
TestSet = TestSet.sample(frac=1).reset_index(drop=True)
TestSet

,Unnamed: 0,taste,customer_review
0,6701,1.0,잘먹엇어요 깔끔하게 와서 좋네요 ㅎㅎ 꽈리고추가 신의 한수였어요 취저네요~~
1,7734,0.0,솔직히 첨에는 배달도 빠르고 맘에 들었는데 오늘 딥빡친다 영수증에도 써있는 시간이 ...
2,5807,1.0,배달 50분 걸린다던데 20분 만에옴;;;;;;;;; 보쌈 양 대박 입니다 앞으로 ...
3,2067,1.0,맛은 괜찮은데 양이 너무.... 배달비도 붙는 거 감안하면 돈 조금 더 내고 일반 ...
4,2059,1.0,오늘 게가 좀 싱싱하지 않아요 슬퍼요
...,...,...,...
900,652,0.0,와우 사진을 못찍었는데 양은 진짜 많았어요 ㅎ 해물탕을 시켯는데 알이랑 곤이 추가추...
901,1725,1.0,괜찮으요ㅛㅛㅛㅛㅛㅛㅛ
902,4293,1.0,배달도 빠르고 항상 맛있게 잘 먹고 있어요~!
903,7856,0.0,다른곳에서 먹어보고 한번 시켜본건데..조리 할 수 없는곳인데 너무 늦게 배달와서 다...


# test01

In [69]:
test01 = TestSet.iloc[0:10]['customer_review'].tolist()
test01

['잘먹엇어요 깔끔하게 와서 좋네요 ㅎㅎ 꽈리고추가 신의 한수였어요 취저네요~~',
 '솔직히 첨에는 배달도 빠르고 맘에 들었는데 오늘 딥빡친다 영수증에도 써있는 시간이 5시3분이지만 받은건 45분 하도 안와서 요기요 고객센터에 연락 할려했더니 드릅게 연락안대고 가게 번호는 가맹점주전용인듯 배달원은 미안하다 한마디 없이 걍 배차 타령 솔직히 죄송합니다 한마디가 어렵나?',
 '배달 50분 걸린다던데 20분 만에옴;;;;;;;;; 보쌈 양 대박 입니다 앞으로 원할머니는 신길점으로 이용할 예정이에요',
 '맛은 괜찮은데 양이 너무.... 배달비도 붙는 거 감안하면 돈 조금 더 내고 일반 해물찜 시키는 게 더 나을 것 같아요 배달도 늦고 맛은 있었어요',
 '오늘 게가 좀 싱싱하지 않아요 슬퍼요',
 '너무맛있었어용 큐알코드 설문 참여했는데 쿠폰은 언제받죠..?',
 '해물찜에는 먹을 해물이 별로 없어요..가격에 비하면 재료가 굉장히 부실하고요, 리뷰를 보니 불맛이 난다하는 말이 어떤건지 이해가 가요, 볶음짬뽕을 먹는기분이에요. 그리고, 볶음밥도 재료를 다 잘게 썰어서 볶은게 아니라 오징어도 통으로 들어가 있어서 그냥 봐도 뭔가 성의가 부족한게 느껴졌고요, 계란찜을 분명 시킨거 같은데 계란찜 물 조절을 못해서 거의 계란국탕이 왔더라구요. 대충 만들었던게 너무 느껴졌어요. 이것들을 거의 3만원돈 내고 먹었는데 너무 후회가 되요.',
 '배달도빨리오고 다 괜창ㅎ은데 고기가 넘 딱딱 ㅜㅜ 고기만 좀 부드러우면 좋겠어요 따듯한 배달 감사용!',
 '배달 시킨지 1시간30분이 지났는데 아직 오지도않네요',
 '맛은 있습니다. 하지만 배달이 세상에 1시간이 넘어서 온걸로만 해도 다시는 주문하고 싶진 않네요']

In [93]:
original = TestSet.iloc[0:10]['taste'].tolist()
original_TasteScore = [int(x) for x in original]
print("true : ", original_TasteScore)

my_review = tfv.transform(test01)
pred = grid_cv.best_estimator_.predict(my_review)
print("pred : ", pred)

error = []
for i in range(10):
  x = original_TasteScore[i]-pred[i]
  error.append(int(x))
print("error: ", error)

print("\n")
print("error 값이 1이면 긍정 레이블링인데, 부정 예측값, error값이 -1이면 부정이였는데 긍정 예측값\n")

num = 0

for i in range(len(error)):
  if error[i] != 0:
    print("error ", error[i], " ", TestSet.iloc[i]['customer_review'])

true :  [1, 0, 1, 1, 1, 1, 0, 1, 0, 1]
pred :  [1. 0. 1. 0. 0. 1. 0. 1. 0. 0.]
error:  [0, 0, 0, 1, 1, 0, 0, 0, 0, 1]


error 값이 1이면 긍정 레이블링인데, 부정 예상, error값이 -1이면 부정이였는데 긍정 예측

error  1   맛은 괜찮은데 양이 너무.... 배달비도 붙는 거 감안하면 돈 조금 더 내고 일반 해물찜 시키는 게 더 나을 것 같아요 배달도 늦고 맛은 있었어요
error  1   오늘 게가 좀 싱싱하지 않아요 슬퍼요
error  1   맛은 있습니다. 하지만 배달이 세상에 1시간이 넘어서 온걸로만 해도 다시는 주문하고 싶진 않네요


In [ ]:
실제 리뷰 내용이 부정인데, 긍정 레이블이 붙어 있는 것도 
학습 모델로 내용 부정 - 레이블 부정 으로 예측 

# test02

In [96]:
test02 = TestSet.iloc[500:510]['customer_review'].tolist()
test02

['맛있다는 리뷰...진짜 진심으로 작성하신거 맞나요? 맛이 하나도 안배여있어요.... 싱거웁다 못해 얌념을 발라놓은것같은 맛이에요... 심지어 추가주문한 볶음밥도 이렇게 맛이 하나도 안배고 맛이없을수가 있나요....? 부침개는 부추조금에 밀가루 부침이고.... 진짜 실망입니다.... 이렇게 맛없는 해물찜은 생전 처음먹어봤어요... 양심적으로 어떻게 이런음식을 팔수있나요? 만드신분 본인이 드셔는 봤나요?',
 '7시쯤에 주문해서 9시가넘어도 안오길래 전화해보니 출발햇다길래 기다렷는데 10시에 전화또해보니 이제와서 주문누락이라하네요 저번에 맛있어서 또시켜먹을라햇는데 이제 시킬일 없을거같네요~',
 '냉면에 면이 너무 붙어서 맛없었어요.. 명란은 너무 조금들어있구요 별로였습니다',
 '맛있어요 나쁘진 않아요',
 '6/20 18시 주문 21시가 되도록 전화 한번 없음 고객센터 통해서 취소 한두번 주문했던 곳도 아닌데 대실망 사과를 받아야겠음',
 '진짜맛있고 양도 많은데 아귀가 별로없어서 아쉬웠어요ㅠㅠ',
 '매일 미더덕찜만 시키다 아귀찜 첨 시켜보는데 이거도 맛있네요~ 찜 종류는 여기서 많이 시켜먹는듯',
 '맛있었어용~~~~추천합니다',
 '해물찜인데. 생각보다 해물종류가 다양하지 않고 짜네요',
 '처음으루 리뷰를 남겨요 저희 큰언니가 요리사라서 입맛이 매우 높아져서 만족을 못햇엇는데 그냥 요리잘하시네요 양도 엄청나고 사장님 복받으실꼬예요']

In [103]:
original = TestSet.iloc[500:510]['taste'].tolist()
original_TasteScore = [int(x) for x in original]
print("true : ", original_TasteScore)

my_review = tfv.transform(test01)
pred = grid_cv.best_estimator_.predict(my_review)
print("pred : ", pred)

error = []
for i in range(10):
  x = original_TasteScore[i]-pred[i]
  error.append(int(x))
print("error: ", error)

print("\n")
print("error 값이  1이면 리뷰에 긍정 레이블링인데, 컴퓨터가 부정으로 예측")
print("error 값이 -1이면 리뷰에 부정 레이블링인데, 컴퓨터가 긍정으로 예측")
print("error 값은 실제 리뷰를 보고 판단해보기\n")

num = 0

for i in range(len(error)):
  if error[i] != 0:
    print("error ", error[i], " ", TestSet.iloc[i]['customer_review'])

true :  [0, 0, 0, 0, 0, 1, 1, 1, 0, 1]
pred :  [1. 0. 1. 0. 0. 1. 0. 1. 0. 0.]
error:  [-1, 0, -1, 0, 0, 0, 1, 0, 0, 1]


error 값이  1이면 리뷰에 긍정 레이블링인데, 컴퓨터가 부정으로 예측
error 값이 -1이면 리뷰에 부정 레이블링인데, 컴퓨터가 긍정으로 예측
error 값은 실제 리뷰를 보고 판단해보기

error  -1   잘먹엇어요 깔끔하게 와서 좋네요 ㅎㅎ 꽈리고추가 신의 한수였어요 취저네요~~
error  -1   배달 50분 걸린다던데 20분 만에옴;;;;;;;;; 보쌈 양 대박 입니다 앞으로 원할머니는 신길점으로 이용할 예정이에요
error  1   해물찜에는 먹을 해물이 별로 없어요..가격에 비하면 재료가 굉장히 부실하고요, 리뷰를 보니 불맛이 난다하는 말이 어떤건지 이해가 가요, 볶음짬뽕을 먹는기분이에요. 그리고, 볶음밥도 재료를 다 잘게 썰어서 볶은게 아니라 오징어도 통으로 들어가 있어서 그냥 봐도 뭔가 성의가 부족한게 느껴졌고요, 계란찜을 분명 시킨거 같은데 계란찜 물 조절을 못해서 거의 계란국탕이 왔더라구요. 대충 만들었던게 너무 느껴졌어요. 이것들을 거의 3만원돈 내고 먹었는데 너무 후회가 되요.
error  1   맛은 있습니다. 하지만 배달이 세상에 1시간이 넘어서 온걸로만 해도 다시는 주문하고 싶진 않네요


# test03

In [51]:
test03 = TestSet.iloc[200:210]['customer_review'].tolist()
test03

['언제 와요?두시간 되가는데 욕나오게 @.',
 '지금 굉장히 화가났구요 11시 50분경에 배달 소요시간 50분이길래 시켰는데 1시15분인 현재까지안왔구요 나가봐야되서 취소를 할려고해도 전화번호가 적혀있길 합니까 환불안받을테니깐 배달안해주셔도되요 비와서 늦는건 이해합니다 배달예정시간을 바꾸시던지요 2번째시켰는데 2번다 늦을거면 배달예정시간을 90분으로 바꾸는게 맞지않나요?',
 '배달이 82분 넘어도오질 않음 도착 예상 45분인데 너무한거 아닌가요',
 '배달완료후 30분이후에왓고 전화번호도없네요 맛을떠나서 최악',
 '다 좋은데, 콜라 주문했는데 안왔어요..업체 번호도 없고 요기요도 안받고 주문할때 너무 복잡하게 하지않는게 좋을것같아요!',
 '요기요 고객센터가 문제인지 셰플리가 문제인지 모르겠네.',
 '예상도착시간은 7시23분인데 7시 50분에 도착 업체 전화번호도 없고 인터넷을 찾아도 없고 고기는 겉만 익히고 속은 익지도 않고 다신 안시킨다',
 '어제 팔다남은 쉬어빠진 딱딱한 밥을 주고 먹으라는 겁니까? 양에 비해 비싼거 감수하고 시키는데 기본이라도 제대로 합시다.',
 '셰플리 자주 배달시켜 먹는데 다시는 이용 안합니다',
 '배달이 안왔는데 왜 완료가 되죠..? 전화도 오지 않습니다..']

In [104]:
original = TestSet.iloc[200:210]['taste'].tolist()
original_TasteScore = [int(x) for x in original]
print("true : ", original_TasteScore)

my_review = tfv.transform(test01)
pred = grid_cv.best_estimator_.predict(my_review)
print("pred : ", pred)

error = []
for i in range(10):
  x = original_TasteScore[i]-pred[i]
  error.append(int(x))
print("error: ", error)

print("\n")
print("error 값이  1이면 리뷰에 긍정 레이블링인데, 컴퓨터가 부정으로 예측")
print("error 값이 -1이면 리뷰에 부정 레이블링인데, 컴퓨터가 긍정으로 예측")
print("error 값은 실제 리뷰를 보고 판단해보기\n")

num = 0

for i in range(len(error)):
  if error[i] != 0:
    print("error ", error[i], " ", TestSet.iloc[i]['customer_review'])

true :  [0, 1, 1, 1, 1, 1, 1, 1, 1, 1]
pred :  [1. 0. 1. 0. 0. 1. 0. 1. 0. 0.]
error:  [-1, 1, 0, 1, 1, 0, 1, 0, 1, 1]


error 값이  1이면 리뷰에 긍정 레이블링인데, 컴퓨터가 부정으로 예측
error 값이 -1이면 리뷰에 부정 레이블링인데, 컴퓨터가 긍정으로 예측
error 값은 실제 리뷰를 보고 판단해보기

error  -1   잘먹엇어요 깔끔하게 와서 좋네요 ㅎㅎ 꽈리고추가 신의 한수였어요 취저네요~~
error  1   솔직히 첨에는 배달도 빠르고 맘에 들었는데 오늘 딥빡친다 영수증에도 써있는 시간이 5시3분이지만 받은건 45분 하도 안와서 요기요 고객센터에 연락 할려했더니 드릅게 연락안대고 가게 번호는 가맹점주전용인듯 배달원은 미안하다 한마디 없이 걍 배차 타령 솔직히 죄송합니다 한마디가 어렵나?
error  1   맛은 괜찮은데 양이 너무.... 배달비도 붙는 거 감안하면 돈 조금 더 내고 일반 해물찜 시키는 게 더 나을 것 같아요 배달도 늦고 맛은 있었어요
error  1   오늘 게가 좀 싱싱하지 않아요 슬퍼요
error  1   해물찜에는 먹을 해물이 별로 없어요..가격에 비하면 재료가 굉장히 부실하고요, 리뷰를 보니 불맛이 난다하는 말이 어떤건지 이해가 가요, 볶음짬뽕을 먹는기분이에요. 그리고, 볶음밥도 재료를 다 잘게 썰어서 볶은게 아니라 오징어도 통으로 들어가 있어서 그냥 봐도 뭔가 성의가 부족한게 느껴졌고요, 계란찜을 분명 시킨거 같은데 계란찜 물 조절을 못해서 거의 계란국탕이 왔더라구요. 대충 만들었던게 너무 느껴졌어요. 이것들을 거의 3만원돈 내고 먹었는데 너무 후회가 되요.
error  1   배달 시킨지 1시간30분이 지났는데 아직 오지도않네요
error  1   맛은 있습니다. 하지만 배달이 세상에 1시간이 넘어서 온걸로만 해도 다시는 주문하고 싶진 않네요
